In [3]:
%load_ext autoreload
%autoreload 2
from art_uw import nl_program, conf_edit
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# import Mintaka data
from create_sample import make_test, make_train
import random
from collections import Counter

sample_config = {
    "section": "superlative",
}
train = make_train('data/mintaka_test.json', full=True)#, **sample_config)
test = make_test('data/mintaka_train.json', full=True)#, **sample_config) #, sample_num=50, uniform=True, 
random_seed = 100
random.Random(4).shuffle(test)


# finding pop level 0 data
test = [ex for ex in test if ex["pop"] == 0]

# add dev set for first run
dev_set = train[50:250]


# filter out pop level = 0 & get its distributions
# pop_distr = dict(Counter([ex["type"] for ex in test if (ex["pop"] == 0)]))
# test = [ex for ex in test if (ex["pop"] == 4) or (ex["pop"] == 5)]

# # make pop 4_5 data the same distribution as pop_0
# filtered_test = []
# for key in pop_distr:
#     section = [ex for ex in test if ex["type"] == key][: 40*pop_distr[key]]
#     filtered_test += section
    
# test = filtered_test
# len(test)
len(train), len(test)


(4000, 151)

In [60]:
# import PopQA data
import dsp
import json
def make_data(path: str, num_sample: int = None) -> dsp.Example:
    data = pd.read_json(path)
    # comment out if full dataset
    if num_sample:
        data = data.sample(n=num_sample, random_state=100)
    final = list(pd.concat([data['question'], data['answer'], data['pop']], axis=1).apply(tuple, axis=1))
    final = [dsp.Example(question=question, answer=json.loads(answer), pop=pop) for question, answer, pop in final]
    return final

full_pop_qa = make_data("data/popQA_low_pop.json")

# make the first 500 in popqa
random.Random(100).shuffle(full_pop_qa)

train = full_pop_qa[:50]
test = full_pop_qa[50:]

# add dev set for first run
dev_set = make_data("data/popQA_mid_pop.json", 200)

test[652], len(test)

({'question': 'What sport does 1988–89 FA Cup Qualifying Rounds play?',
  'answer': ['association football', 'football', 'soccer'],
  'pop': 1},
 1349)

In [62]:
# Entity Questions data
import random
import numpy as np
import dsp
import os

path = "data/entity_ques/test"
files = os.listdir(path)

def entity_question_data(n_sample: int = None, thres: int = 1):
    final = []
    
    for f in files:
        df = pd.read_json('/'.join([path, f]))
        df["pop_bin"] = np.log10(df["q_pop"].apply(lambda x: x[0]+1 if x[0] else None))
        df = df.dropna()
        df["pop_bin"] = df["pop_bin"].apply(int)
        # filter low pop data
        data = df[df["pop_bin"]<=thres]
        
        if n_sample:
            true_n = n_sample if len(data) >= n_sample else len(data)
            data = data.sample(n=true_n, random_state=100)
        
        section = list(pd.concat([data['question'], data['answers']], axis=1).apply(tuple, axis=1))
        section = [dsp.Example(question=question, answer=answer) for question, answer in section]
        final += section
        
    return final

data = entity_question_data()
random.Random(100).shuffle(data)
train, test = data[:100], data[100:]

# add dev set for first run
dev_set = data[1100:1600]

len(test)

5712

In [18]:
model_list = [
#     "text-davinci-002", 
#     "text-davinci-003", 
#     "gpt-3.5-turbo-instruct-0914",
#     "gpt-3.5-turbo-0301", 
    "gpt-3.5-turbo-0613", 
#     "gpt-3.5-turbo", 
#     "gpt-4o",
    "gpt-4",
#     "gpt-4-0613",
#     "gpt-4-turbo",
             ]


# error_ques = [2, 33, 41, 46, 81, 129, 141]
# test = [test[i] for i in error_ques]


for name in model_list:
    print(f"Running test on {name}, dataset size is {len(test)}...")
    nl_program(train[:3], test, section="mintaka_ablation_SE", model_name=name, 
               get_log=False, echo=False, self_eval=True)


Running test on gpt-3.5-turbo-0613, dataset size is 151...
Create an empty list in json file!
Caching the llm selected similar tasks!


0it [00:00, ?it/s]

This is the injected eval result: 
Make sure you follow the following instructions before you move on. Use [Internal Knowledge] - This question can be answered based on existing knowledge about the Dune series by Frank Herbert. The last book written by Frank Herbert in the original Dune series is "Chapterhouse: Dune." DO NOT use [search], [code generate], [code execute], [code edit], [string operations], [arithmetic]. Keep using the right tools until you reach a final answer that is reliable.
Self eval injected successfully!


1it [00:10, 10.19s/it]

This is the injected eval result: 
Make sure you follow the following instructions before you move on. Tools are necessary. You should use [code generate], [code execute]. DO NOT use [search], [code edit], [string operations], [arithmetic]. Keep using the right tools until you reach a final answer that is reliable.
Self eval injected successfully!


...

149it [43:05, 15.98s/it]

This is the injected eval result: 
Make sure you follow the following instructions before you move on. Use [Internal Knowledge] only. You should not use [generate python code], [code execute], [search], [code edit], [string operations], [arithmetic]. Keep using the right tools until you reach a final answer that is reliable.
Self eval injected successfully!


150it [43:28, 18.00s/it]

This is the injected eval result: 
Make sure you follow the following instructions before you move on. Use your [Internal Knowledge] to answer the question. DO NOT use [search], [code generate], [code execute], [code edit], [string operations], [arithmetic]. Keep using the right tools until you reach a final answer that is reliable.
Self eval injected successfully!


151it [43:43, 17.38s/it]


In [20]:
from metric import check_answer, check_answer_new


files = [
#     "mintaka_(ori)_gpt-4_out.json",
#     "mintaka_(verb)_gpt-4_out.json",
#     "mintaka_(eval_v2.0)_gpt-4_out.json",
#     "pop_qa_(ori_v3.0)_gpt-4_out.json",
#     "pop_qa_(eval_v3.0)_gpt-4_out.json",
#     "pop_qa_(verb)_gpt-3.5-turbo_out.json",
#     "mintaka_(verb)_gpt-3.5-turbo_out.json",
#     "whole_data_uncalibrated/full_pop_0_text-davinci-002_out.json",
#     "whole_data_uncalibrated/full_pop_0_text-davinci-003_out.json",
    "mintaka_ablation_gpt-3.5-turbo-0613_out.json",
    "mintaka_ablation_gpt-4_out.json",
    "mintaka_ablation_CPC_gpt-3.5-turbo-0613_out.json",
    "mintaka_ablation_CPC_gpt-4_out.json",
    
]
for f in files:
    df = pd.read_json(f)
    df["adj_correct"] = df.apply(check_answer, axis=1)
    acc_old = sum(df["adj_correct"]) / len(df["adj_correct"]), len(df)
    print(acc_old)


# arr = np.array(sub_ind)
# corr_ind = df.loc[df["adj_correct"]==True].index
# arr = arr[corr_ind]

# df2 = pd.read_json("pop_qa_(eval)_gpt-4_out.json")
# df = df.drop('adj_correct', axis=1)
# df2.iloc[arr] = df.iloc[corr_ind]

# df2.iloc[arr]
# df2["adj_correct"] = df2.apply(check_answer, axis=1)
# sum(df2["adj_correct"]) / len(df2["adj_correct"]), len(df2)


# print(df.iloc[corr_ind]["after_art"].values[0])



(0.5364238410596026, 151)
(0.5761589403973509, 151)
(0.5364238410596026, 151)
(0.5761589403973509, 151)
